In [2]:
# load environment variables from .env file (requires `python-dotenv`)
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Translate the following from English into {language}"),
    ("user", "{text}")
])


llm = ChatOpenAI(
    model="llama-3.1-8b-instant",
    base_url="https://api.groq.com/openai/v1",
)

parser = StrOutputParser()

chain = prompt_template | llm | parser

print(chain.invoke({"text": "nice to meet you", "language": "Chinese"}))



您好（nín hǎo）


In [4]:
analysis_prompt = ChatPromptTemplate.from_template("我应该怎么回答这句话？{talk}。给我一个五个字以内的示例")

chain2 = {"talk": chain} | analysis_prompt | llm | parser

print(chain2.invoke({"text": "nice to meet you", "language": "Chinese"}))

"很高兴见到你"


In [5]:
from langchain_core.runnables import RunnableMap, RunnableLambda

prompt_template_zh = ChatPromptTemplate.from_messages([
    ("system", "Translate the following from English into Chinese"),
    ("user", "{text}")
])

prompt_template_fr = ChatPromptTemplate.from_messages([
    ("system", "Translate the following from English into French"),
    ("user", "{text}")
])

# 构建链
chain_zh = prompt_template_zh | llm | parser
chain_fr = prompt_template_fr | llm | parser

# 并行执行两个链
parallel_chains = RunnableMap({
    "zh_translation": chain_zh,
    "fr_translation": chain_fr,
})

# 合并结果
final_chain = parallel_chains | RunnableLambda(lambda x: f"Chinese: {x['zh_translation']}\nFrench: {x['fr_translation']}")

print(final_chain.invoke({"text": "nice to meet you"}))

Chinese: 很高興见到你
French: Enchanté(e).


In [10]:
final_chain.get_graph().print_ascii()

   +----------------------------------------------+    
   | Parallel<zh_translation,fr_translation>Input |    
   +----------------------------------------------+    
                  ***             ***                  
                **                   **                
              **                       **              
+--------------------+          +--------------------+ 
| ChatPromptTemplate |          | ChatPromptTemplate | 
+--------------------+          +--------------------+ 
           *                               *           
           *                               *           
           *                               *           
    +------------+                  +------------+     
    | ChatOpenAI |                  | ChatOpenAI |     
    +------------+                  +------------+     
           *                               *           
           *                               *           
           *                               *    

## 保存聊天记录

In [11]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

history.add_user_message("你是谁？")
ai_message = llm.invoke(history.messages)
print(ai_message.content)
history.add_message(ai_message)

history.add_user_message("请重复一次")
ai_message2 = llm.invoke(history.messages)
print(ai_message2.content)
history.add_message(ai_message2)

我是一个人工智能模型，我被称为Chatbot或语言模型。我的任务是帮助用户交谈、回答问题和提供信息。我可以学习和改进我的知识以更好地满足你的需求。
我是一个人工智能模型，能够理解和回答问题。我的工作是提供信息、帮助解决问题和进行互动。


In [6]:
from langchain_redis import RedisChatMessageHistory

history = RedisChatMessageHistory(session_id="test", redis_url="redis://localhost:6379/0")

# history.add_user_message("你是谁？")
# ai_message = llm.invoke(history.messages)
# print(ai_message.content)
# history.add_message(ai_message)

history.add_user_message("请重复一次")
ai_message2 = llm.invoke(history.messages)
print(ai_message2.content)
history.add_message(ai_message2)

21:02:28 redisvl.index.index INFO   Index already exists, not overwriting.
21:02:29 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
我是一個語言模型，我被稱為 LLaMA。我的功能是用自然語言和人類類似的語句來回答你提出的問題，或者和你進行對話。


## 3、聊天消息整合 LCEL

In [7]:
from langchain_core.runnables.history import RunnableWithMessageHistory

runnable = RunnableWithMessageHistory(
    llm,
    get_session_history=lambda: history,
)

history.clear()

runnable.invoke({"text": "你是谁？"})
runnable.invoke({"text": "请重复一次"})

21:07:54 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
21:07:54 httpx INFO   HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='我是一個語言模型 llamá, 我被設計用來回答你們的問題、聊天、甚至創作文本。我的能力還在不斷學習和改進中，現在可以回答關於各種主題的問題。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 113, 'total_tokens': 168, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.21656186700000002, 'prompt_time': 0.006616617, 'completion_time': 0.073333333, 'total_time': 0.07994995}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'id': 'chatcmpl-eeb2cdc1-97c8-43b6-851a-c91b627028a2', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--887686f8-0d0e-42b9-b6b5-c181f8221e77-0', usage_metadata={'input_tokens': 113, 'output_tokens': 55, 'total_tokens': 168, 'input_token_details': {}, 'output_token_details': {}})